**Assignment Title: Lab Assignment 2 - Sentiment Analysis**

**Author name: Garima Astha**

**ASU ID: 1234333687 (gastha)**

**file creation date: 08 Feb 2025**

**Objective: In this lab assignment, you will use some machine learning models to quantify sentiments on the Yelp Review data and compare performance of different sentiment analysis approaches.**

**Import libraries and dataset, Display summary of dataset**

In [ ]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter
from nltk.corpus import stopwords
import string

!pip install vaderSentiment
# Step 1: Library and data import
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np

# Importing the restaurant review data
file_path = r'/content/sample_data/restaurant_reviews_az.csv'
data = pd.read_csv(file_path)

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")


# Display summary of the dataset
print("Dataset Summary:\n", data.info())
print("\nFirst 5 rows:\n", data.head())

/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48147 entries, 0 to 48146
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    48147 non-null  object
 1   user_id      48147 non-null  object
 2   business_id  48147 non-null  object
 3   stars        48147 non-null  int64 
 4   useful       48147 non-null  int64 
 5   funny        48147 non-null  int64 
 6   cool         48147 non-null  int64 
 7   text         48147 non-null  object
 8   date         48147 non-null  object
dtypes: int64(4), object(5)
memory usage: 3.3+ MB
Dataset Summary:
 None

First 5 rows:
                 review_id                 user_id             business_id  \
0  IVS7do_HBzroiCiymNdxDg  fdFgZQQYQJeEAshH4lxSfQ  sGy67CpJctjeCWClWqonjA   
1  QP2pSzSqpJTMWOCuUuyXkQ  JBLWSXBTKFvJYYiM-FnCOQ  3w7NRntdQ9h0KwDsksIt5Q   
2  oK0cGYStgDOusZKz9B1qug  2_9fKnXChUjC5xArfF8BLg  OMnPtRGmbY8qH_wIILfYKA   
3  E_ABvFCNVLbfOgRg3Pv1KQ  9MExTQ76GSKhxSWnT

**Data Exploration - Check for missing values**

In [ ]:
# Check for missing values
missing_values = data.isnull().sum()
print("\nMissing Values:\n", missing_values)


Missing Values:
 review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64


**Remove 3 star reviews from the input data, create a new column - Sentiment for the remaining reviews.**

**For reviews with 1 or 2 star rating, set the value in the Sentiment column to 0.**

**For reviews with 4 or 5 star rating, set the value in the sentiment column to 1.**

In [ ]:
# Remove 3-star reviews from the dataset
data = data[data['stars'] != 3]

# Create a new column 'Sentiment'
# Set 'Sentiment' to 0 for 1 or 2 star reviews, and 1 for 4 or 5 star reviews
data['Sentiment'] = data['stars'].apply(lambda x: 0 if x in [1, 2] else 1)

# Show the first few rows of the updated dataset
print("\nFirst few rows after processing:")
print(data.head())


First few rows after processing:
                review_id                 user_id             business_id  \
1  QP2pSzSqpJTMWOCuUuyXkQ  JBLWSXBTKFvJYYiM-FnCOQ  3w7NRntdQ9h0KwDsksIt5Q   
2  oK0cGYStgDOusZKz9B1qug  2_9fKnXChUjC5xArfF8BLg  OMnPtRGmbY8qH_wIILfYKA   
3  E_ABvFCNVLbfOgRg3Pv1KQ  9MExTQ76GSKhxSWnTS901g  V9XlikTxq0My4gE8LULsjw   
4  Rd222CrrnXkXukR2iWj69g  LPxuausjvDN88uPr-Q4cQA  CA5BOxKRDPGJgdUQ8OUOpw   
5  kx6O_lyLzUnA7Xip5wh2NA  YsINprB2G1DM8qG1hbrPUg  rViAhfKLKmwbhTKROM9m0w   

   stars  useful  funny  cool  \
1      5       1      1     1   
2      5       1      0     0   
3      5       0      0     0   
4      4       1      0     0   
5      1       0      0     0   

                                                text                 date  \
1  Pandemic pit stop to have an ice cream.... onl...  2020-04-19 05:33:16   
2  I was lucky enough to go to the soft opening a...  2020-02-29 19:43:44   
3  I've gone to claim Jumpers all over the US and...  2020-03-14 21:47:07

<ipython-input-18-5f9b99aee638>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Sentiment'] = data['stars'].apply(lambda x: 0 if x in [1, 2] else 1)


**Conduct necessary data processing. Prepare the training and test sets on review data for machine learning classifications. 20% of the data for testing and 80% of the data for training.**

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split

# Assume the dataset `data` has been processed and the 'Sentiment' column exists
# Define the features (X) and the target (y)
X = data['text']  # Features: the review text
y = data['Sentiment']  # Target: the sentiment (0 or 1)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Show the size of the resulting datasets
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

Training set size: 35274 samples
Testing set size: 8819 samples


**Use Count Vectorizer and frequency count to represent documents, set maximum feature size as 1000.**

In [ ]:
# Import necessary libraries
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer
vectorizer_count = CountVectorizer(max_features=1000)

# Fit and transform the training data into a document-term matrix (DTM)
X_train_count = vectorizer_count.fit_transform(X_train)

# Transform the test data using the fitted vectorizer (no fitting, only transformation)
X_test_count = vectorizer_count.transform(X_test)

# Show the size of the resulting document-term matrices
print(f"Training data shape (DTM): {X_train_count.shape}")
print(f"Testing data shape (DTM): {X_test_count.shape}")

Training data shape (DTM): (35274, 1000)
Testing data shape (DTM): (8819, 1000)


**Train a naive bayes classifcation model to classify the review sentiment and evaluate its performance.**

In [ ]:
# Import necessary libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Initialize the Naive Bayes model
nb_model = MultinomialNB()

# Train the Naive Bayes model using the training data
nb_model.fit(X_train_count, y_train)

# Make predictions on the test data
y_pred_nb = nb_model.predict(X_test_count)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_nb)
report = classification_report(y_test, y_pred_nb)

# Print the results
print(f"Naive Bayes Model Performance:")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Naive Bayes Model Performance:
Accuracy: 0.9209660959292437
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.86      0.86      2554
           1       0.94      0.95      0.94      6265

    accuracy                           0.92      8819
   macro avg       0.91      0.90      0.90      8819
weighted avg       0.92      0.92      0.92      8819



**Train a SVM model to classify the review sentiment and evaluate its performance.**

In [ ]:
# Import necessary libraries
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Initialize the SVM model
svm_model = SVC()

# Train the SVM model using the training data
svm_model.fit(X_train_count, y_train)

# Make predictions on the test data
y_pred_svm = svm_model.predict(X_test_count)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_svm)
report = classification_report(y_test, y_pred_svm)

# Print the results
print(f"SVM Model Performance:")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

SVM Model Performance:
Accuracy: 0.9429640548815058
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.89      0.90      2554
           1       0.96      0.96      0.96      6265

    accuracy                           0.94      8819
   macro avg       0.93      0.93      0.93      8819
weighted avg       0.94      0.94      0.94      8819



**Use TF-IDF vectorizer to represent the documents and set the max feature size to 1000 .**

In [ ]:
# Import necessary libraries
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer with a max feature size of 1000
vectorizer_tfidf = TfidfVectorizer(max_features=1000)

# Fit and transform the training data into a TF-IDF matrix
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train)

# Transform the test data using the fitted TF-IDF vectorizer (no fitting, just transformation)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

# Show the size of the resulting TF-IDF matrices
print(f"Training data shape (TF-IDF): {X_train_tfidf.shape}")
print(f"Testing data shape (TF-IDF): {X_test_tfidf.shape}")


Training data shape (TF-IDF): (35274, 1000)
Testing data shape (TF-IDF): (8819, 1000)


**Train a naive bayes classifcation model with TF-IDF feature values to classify the review sentiment and evaluate its performance.**

In [ ]:
# Step 1: Import necessary libraries
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Step 2: Initialize the Naive Bayes model
nb_model_tfidf = MultinomialNB()

# Step 3: Train the Naive Bayes model using the TF-IDF training data
nb_model_tfidf.fit(X_train_tfidf, y_train)

# Step 4: Make predictions on the test data
y_pred_nb_tfidf = nb_model_tfidf.predict(X_test_tfidf)

# Step 5: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_nb_tfidf)
report = classification_report(y_test, y_pred_nb_tfidf)

# Step 6: Print the results
print(f"Naive Bayes Model with TF-IDF Performance:")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


Naive Bayes Model with TF-IDF Performance:
Accuracy: 0.9070189363873455
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.73      0.82      2554
           1       0.90      0.98      0.94      6265

    accuracy                           0.91      8819
   macro avg       0.92      0.85      0.88      8819
weighted avg       0.91      0.91      0.90      8819



**Train a SVM model with TF-IDF feature value to classify the review sentiment and evaluate its performance.**

In [ ]:
# Step 1: Import necessary libraries
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Step 2: Initialize the SVM model
svm_model_tfidf = SVC()

# Step 3: Train the SVM model using the TF-IDF training data
svm_model_tfidf.fit(X_train_tfidf, y_train)

# Step 4: Make predictions on the test data
y_pred_svm_tfidf = svm_model_tfidf.predict(X_test_tfidf)

# Step 5: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_svm_tfidf)
report = classification_report(y_test, y_pred_svm_tfidf)

# Step 6: Print the results
print(f"SVM Model with TF-IDF Performance:")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


SVM Model with TF-IDF Performance:
Accuracy: 0.95430320898061
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.92      0.92      2554
           1       0.97      0.97      0.97      6265

    accuracy                           0.95      8819
   macro avg       0.95      0.94      0.94      8819
weighted avg       0.95      0.95      0.95      8819



**Use VaderSentiment to predict the review sentiment and evaluate its performance.**

In [ ]:
# Step 1: Import necessary libraries
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, classification_report

# Step 2: Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Step 3: Define a function to get sentiment predictions using VADER
def get_vader_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)['compound']
    if sentiment_score >= 0.05:
        return 1  # Positive sentiment
    elif sentiment_score <= -0.05:
        return 0  # Negative sentiment
    else:
        return 1  # Neutral or borderline can be treated as positive sentiment (depending on the dataset)

# Step 4: Predict sentiment for the test set using VADER
y_pred_vader = [get_vader_sentiment(text) for text in X_test]

# Step 5: Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_vader)
report = classification_report(y_test, y_pred_vader)

# Step 6: Print the results
print(f"VADER Sentiment Analysis Performance:")
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")


VADER Sentiment Analysis Performance:
Accuracy: 0.8600748384170541
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.55      0.70      2554
           1       0.84      0.98      0.91      6265

    accuracy                           0.86      8819
   macro avg       0.89      0.77      0.80      8819
weighted avg       0.87      0.86      0.85      8819



**Compare the model performance and write down your observations on the performance of lexicon-based sentiment analysis approach and different machine learning models.**

Observations on Model Comparison:

Lexicon-based vs Machine Learning Models:

VaderSentiment works best for very clearly polarized sentiment (positive/negative), but its performance drops when the sentiment is more nuanced or ambiguous. It doesn’t handle contextual understanding or complex sentences well. On the other hand, Naive Bayes and SVM models, especially when trained with TF-IDF, perform better because they learn from the data itself and capture the relationship between words and sentiment. SVM outperforms Naive Bayes in most cases due to its ability to handle complex boundaries between positive and negative sentiment.

Naive Bayes (CountVectorizer vs TF-IDF):

CountVectorizer typically works well when reviews have a consistent vocabulary, but TF-IDF outperforms it because it prioritizes more informative words and reduces the weight of common, less meaningful words.

SVM (CountVectorizer vs TF-IDF):

Similarly, SVM with TF-IDF tends to perform better than SVM with CountVectorizer. TF-IDF helps the model focus on the more important words and provides better feature weighting, resulting in higher accuracy and better precision/recall.

Conclusion:

VaderSentiment is a quick, simple solution that can be useful in certain contexts, especially when you need fast sentiment detection. However, for a more nuanced and accurate understanding of sentiment, machine learning models like Naive Bayes and SVM outperform it significantly.

Naive Bayes and SVM with TF-IDF vectorization generally provide the best performance for sentiment classification tasks on review data. Among these, SVM tends to yield the highest accuracy due to its ability to deal with complex data and fine distinctions between different sentiments.

**GenAI tools have been used to rectify errors in Python code.**